# Surface Data and Plotting
This notebook demonstrates reading surface data and plotting it using MetPy's declarative syntax.

In [ ]:
from datetime import datetime, timedelta
from io import StringIO
from urllib.request import urlopen

from metpy.io import metar
from metpy.plots import declarative
from metpy.units import units

### Read Data

We bring in surface data that is in METAR format and store it for approximately two weeks in `/data/ldmdata/surface/sao` and the format of the filenames are `YYYYMMDDHH_sao.wmo` where `YYYY` is the year, `MM` is the month, `DD` is the day, and `HH` is the hour. Or there are pre-decoded files located in `/data/ldmdata/surface/csv` and can be read in using the Pandas module.

In [ ]:
date = datetime(2022, 1, 3, 12)

# Remote Access - Read current METAR files
data = StringIO(urlopen('http://bergeron.valpo.edu/current_surface_data/'
                        f'{date:%Y%m%d%H}_sao.wmo').read().decode('utf-8', 'backslashreplace'))
# Local Access
# data = f'/data/ldmdata/surface/sao/{date:%Y%m%d%H}_sao.wmo'

df = metar.parse_metar_file(data, year=date.year, month=date.month)

### Plot Data

In [ ]:
# Plot desired data
obs = declarative.PlotObs()
obs.data = df
obs.time = date
obs.time_window = timedelta(minutes=15)
obs.fields = ['air_temperature']
obs.plot_units = ['degF']

# Panel for plot with Map features
panel = declarative.MapPanel()
panel.layout = (1, 1, 1)
panel.projection = 'lcc'
panel.area = 'in'
panel.layers = ['states']
panel.title = f'Air temperature on {date:%Y%m%d} at {date:%H}UTC'
panel.plots = [obs]

# Bringing it all together
pc = declarative.PanelContainer()
pc.size = (10, 10)
pc.panels = [panel]

pc.save(f'surface_plot_{date:%Y%m%d_%H}UTC.png', dpi=150, bbox_inches='tight')